In [9]:
import pandas as pd
import nltk
from nltk.stem import SnowballStemmer
from pattern.es import parsetree
import re
from collections import defaultdict
import numpy as np
import unidecode
from textblob import TextBlob
import time
#from spellchecker import SpellChecker
#from googletrans import Translator
#from translate import Translator
from classifier import SentimentClassifier as sf
clf = sf()

#translator = Translator()
#translator = Translator("spanish", "english")

#spell = SpellChecker(language='es', distance=2) # Spanish dictionary
#spell.word_frequency.load_words(['jubilación', 'desahucio'])

palabras_funcionales = nltk.corpus.stopwords.words("spanish")    
stemmer = SnowballStemmer('spanish')

In [10]:
data = pd.read_csv("Data\\preguntas.csv", sep=',', encoding = "ISO-8859-1")
data.head(3)

,Pregunta,respuesta,Tema,Pers/empresa,Polarity,Nota,ambiguedad
0,Buenos días. Mi nombre es Oscar Castro. Ayer ...,NaN,Contacto,NaN,0,NaN,NaN
1,quiero saber si ustedes tratan de temas labor...,NaN,Renuncia/Despido/Desahucio,NaN,NaN,NaN,NaN
2,se puede jubilar con 28 años de aporte y 61 d...,NaN,Jubilacion Patronal,NaN,NaN,NaN,NaN


In [11]:
try:
    data = pd.concat(g for _, g in data.groupby(["Pregunta",'Tema']) if len(g) > 1)
    print(data)
except:
    print('no duplicates')

no duplicates


In [4]:
data.Tema.value_counts()

Jubilacion Patronal             227
IESS                            165
Consultoria                     117
No Topic                         90
Renuncia/Despido/Desahucio       82
Greeting                         62
Contacto                         56
Hi Five                          44
job seeker                       43
Otros servicios                  43
Charlas/Capacitaciones           33
Facturacion/Retencion/Cobros     25
Queja                            22
Name: Tema, dtype: int64

In [5]:
data['target'] = data.Tema.map({'Jubilacion Patronal': 0, 'Consultoria':1, 'Renuncia/Despido/Desahucio':2, 'IESS':3, 
                                'Greeting':4, 'Contacto':5, 'No Topic':6, 'Queja':7, 'Otros servicios':8, 
                                    'Charlas/Capacitaciones':9, 'Hi Five':10, 'job seeker':11, 'Facturacion/Retencion/Cobros':12})

In [6]:
#def spell_check(texto):
#    tokens = nltk.word_tokenize(texto, "spanish")
#    corr_tokens = []
    
#    for token in tokens: 
#        if spell[token]:
#            corr_tokens.append(token)
#        else:
#            corr_tokens.append(spell.correction(token))
       
#    texto2 = " ".join(str(item) for item in corr_tokens)
#    return texto2


def hasNumbers(string):
    return bool(re.search(r'\d', string))


def hasBC(string):
    i = string.find('/')
    return bool(i != -1)


def other_check(token):    
    b1 = not hasNumbers(token)
    b2 = not hasBC(token)
    return (b1 and b2)

def remove_accent(word):
    return unidecode.unidecode(word)

def stem_lemma(word):
    stemmer = SnowballStemmer('spanish')
    word = parsetree(word, lemmata=True)[0].lemmata[0]
    word = stemmer.stem(word) 
    return word


def token_and_clean(texto):
    try:          
        tokens = nltk.word_tokenize(texto, "spanish")
        token_list = []
        for token in tokens:        
            if token not in palabras_funcionales:
                token = stem_lemma(token)
                token = remove_accent(token)
                if len(token) >= 2 and other_check(token):
                    token_list.append(token)
                
        if not len(token_list) > 0:
            print('No Tokens in: {0}'.format(texto))
        return token_list
    except:
        print('error en el texto: {0}'.format(texto))        

                
    
def analyze_token(texto, tema, tokens_limpios):
    try:
        token_list = token_and_clean(texto)
        for t in token_list:
            t_list = tokens_limpios[t]
            t_list[0] += 1
            t_list[int(tema + 1)] += 1
            tokens_limpios[t] = t_list
        return tokens_limpios
    except:
        print('error en el texto: {0}'.format(texto))

        
def vectorize_phrase(texto, vocab):
    tokens = token_and_clean(texto)
    vector = np.zeros(len(vocab))
    for t in tokens:
        if t in vocab:
            vector[vocab.index(t)] = 1
    return vector
    
    
def n_token(sentence):
    token_list = token_and_clean(sentence) 
    return len(token_list) 


def polarity(message): #blob has a limit on api calls
    #blob = TextBlob(message)
    
    try:
        #leng = blob.detect_language()
        #leng = 'es'
        
        #if leng != 'en':
            #blob = blob.translate(to='en').lower() 
            #traduccion = translator.translate(message)
            #blob = TextBlob(traduccion.text).lower()
            #blob = TextBlob(traduccion).lower()
        #else:
            #blob = blob.lower() 
            
        pol = clf.predict(message)
    
    except Exception as e:
        print('Exception en polarity: {0}'.format(e))
        pol = 0
        
            
    #pol = blob.sentiment[0]
    
    return pol


def percent_greet(sentence):
    tgreet = ['hol', 'buen', 'tard', 'dia', 'noch']
    count = 0
    tokens = token_and_clean(sentence)
    for w in tokens:       
        if w in tgreet:
            count += 1  
    if len(tokens) > 0:   
        return count/len(tokens) 
    else:
        return 0
    
def ex_capac(sentence):
    tcap = ['charl', 'curs', 'capacit', 'seminari', 'formacion', 'capacitacion']
    count = 0
    tokens = token_and_clean(sentence)
    for w in tokens:       
        if w in tcap:
            count += 1  
    return (count > 0)*1
    
    


In [7]:
tokens_limpios =  defaultdict(lambda: np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])) 

In [8]:
for sent, tema in zip(data.Pregunta, data.target):
    #sent = spell_check(sent)
    tokens_limpios = analyze_token(sent, tema, tokens_limpios)

No Tokens in: cuando


In [9]:
tokens_limpios['para']

array([2, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [10]:
for key in tokens_limpios:    
    mlista = tokens_limpios[key]
    if np.argmax(mlista[1:]) != 6:
        mlista = np.append(mlista[0], np.max(mlista[1:]/mlista[0]))
    else:
        mlista = np.array([0,0])
    tokens_limpios[key] = mlista  

counts = 3
f_tokens = {k: v for k, v in tokens_limpios.items() if v[0] >= counts and v[1] > 1/v[0]}
f_tokens

{'buen': array([295.        ,   0.33559322]),
 'dia': array([163.        ,   0.26993865]),
 'mi': array([30. ,  0.6]),
 'nombr': array([19.        ,  0.26315789]),
 'llam': array([16. ,  0.5]),
 'empres': array([125.  ,   0.44]),
 'me': array([60.        ,  0.26666667]),
 'hoy': array([7.        , 0.28571429]),
 'ser': array([79.        ,  0.36708861]),
 'posibl': array([12. ,  0.5]),
 'contact': array([20.  ,  0.55]),
 'quer': array([97.        ,  0.30927835]),
 'sab': array([90.        ,  0.27777778]),
 'si': array([139.        ,   0.31654676]),
 'usted': array([42.        ,  0.30952381]),
 'trat': array([5. , 0.4]),
 'tem': array([26.        ,  0.30769231]),
 'laboral': array([24.        ,  0.29166667]),
 'empleador': array([10. ,  0.5]),
 'sin': array([10. ,  0.4]),
 'emple': array([31.        ,  0.32258065]),
 'renunci': array([28.        ,  0.60714286]),
 'pod': array([236.        ,   0.33474576]),
 'jubil': array([219.        ,   0.48858447]),
 'ano': array([274.        ,   0.51

In [11]:
vocab = list(f_tokens.keys())
print(len(vocab))
np.save('Data\\vocab.npy', vocab)

572


In [12]:
X = vectorize_phrase(data.Pregunta[0], vocab)
for sent in data.Pregunta[1:]:
    vector = vectorize_phrase(sent, vocab)
    X = np.vstack((X, vector))    

No Tokens in: cuando


In [13]:
X.shape

(1009, 572)

In [14]:
#tot_sum = np.sum(X, axis =1)
#X = np.column_stack((X, tot_sum))

In [15]:
nt = [n_token(x) for x in data.Pregunta]
X = np.column_stack((X, nt))

No Tokens in: cuando


In [16]:
X.shape

(1009, 573)

In [17]:
pg = [percent_greet(x) for x in data.Pregunta]
X = np.column_stack((X, pg))

No Tokens in: cuando


In [18]:
cap = [ex_capac(x) for x in data.Pregunta]
X = np.column_stack((X, cap))

No Tokens in: cuando


In [19]:
X.shape

(1009, 575)

In [20]:
pol = []    

In [21]:
counter = 0
for sent in data.Pregunta:
    #time.sleep(0.05)
    p = polarity(sent)
    pol.append(p)
    print('{0} of {1}'.format(counter+1,len(data.Pregunta)))
    
    counter+=1

#pol =  np.load('Data\\pol.npy', allow_pickle=True)

1 of 1009
2 of 1009
3 of 1009
4 of 1009
5 of 1009
6 of 1009
7 of 1009
8 of 1009
9 of 1009
10 of 1009
11 of 1009
12 of 1009
13 of 1009
14 of 1009
15 of 1009
16 of 1009
17 of 1009
18 of 1009
19 of 1009
20 of 1009
21 of 1009
22 of 1009
23 of 1009
24 of 1009
25 of 1009
26 of 1009
27 of 1009
28 of 1009
29 of 1009
30 of 1009
31 of 1009
32 of 1009
33 of 1009
34 of 1009
35 of 1009
36 of 1009
37 of 1009
38 of 1009
39 of 1009
40 of 1009
41 of 1009
42 of 1009
43 of 1009
44 of 1009
45 of 1009
46 of 1009
47 of 1009
48 of 1009
49 of 1009
50 of 1009
51 of 1009
52 of 1009
53 of 1009
54 of 1009
55 of 1009
56 of 1009
57 of 1009
58 of 1009
59 of 1009
60 of 1009
61 of 1009
62 of 1009
63 of 1009
64 of 1009
65 of 1009
66 of 1009
67 of 1009
68 of 1009
69 of 1009
70 of 1009
71 of 1009
72 of 1009
73 of 1009
74 of 1009
75 of 1009
76 of 1009
77 of 1009
78 of 1009
79 of 1009
80 of 1009
81 of 1009
82 of 1009
83 of 1009
84 of 1009
85 of 1009
86 of 1009
87 of 1009
88 of 1009
89 of 1009
90 of 1009
91 of 1009
92 of 10

695 of 1009
696 of 1009
697 of 1009
698 of 1009
699 of 1009
700 of 1009
701 of 1009
702 of 1009
703 of 1009
704 of 1009
705 of 1009
706 of 1009
707 of 1009
708 of 1009
709 of 1009
710 of 1009
711 of 1009
712 of 1009
713 of 1009
714 of 1009
715 of 1009
716 of 1009
717 of 1009
718 of 1009
719 of 1009
720 of 1009
721 of 1009
722 of 1009
723 of 1009
724 of 1009
725 of 1009
726 of 1009
727 of 1009
728 of 1009
729 of 1009
730 of 1009
731 of 1009
732 of 1009
733 of 1009
734 of 1009
735 of 1009
736 of 1009
737 of 1009
738 of 1009
739 of 1009
740 of 1009
741 of 1009
742 of 1009
743 of 1009
744 of 1009
745 of 1009
746 of 1009
747 of 1009
748 of 1009
749 of 1009
750 of 1009
751 of 1009
752 of 1009
753 of 1009
754 of 1009
755 of 1009
756 of 1009
757 of 1009
758 of 1009
759 of 1009
760 of 1009
761 of 1009
762 of 1009
763 of 1009
764 of 1009
765 of 1009
766 of 1009
767 of 1009
768 of 1009
769 of 1009
770 of 1009
771 of 1009
772 of 1009
773 of 1009
774 of 1009
775 of 1009
776 of 1009
777 of 1009
778 

In [22]:
#np.save('Data\\pol.npy', pol)

In [23]:
X = np.column_stack((X, pol))

In [24]:
X.shape

(1009, 576)

In [25]:
np.save('Data\\X.npy', X)

In [26]:
#X2 = np.load('X.npy', allow_pickle=True)


In [27]:
y = data.target.to_numpy()

In [28]:
np.save('Data\\y.npy', y)